# `folium` IPython Magic Demo

To try to simplify embedding maps in Jupyer notebooks, we can create some magic to load a map based on location, provided as latitude / longitude co-ordinates.

(If no latitude and longitude values are provided, a default value is used.)

In [1]:
%load_ext folium_magic 

In [2]:
%folium_map -l 52.0250,-0.7084

#### Adding Markers

The magic can be extended to allow for the inclusion of a marker:

In [3]:
%folium_map -m 52.0250,-0.7084,"My marker" -z 12

If the output of the previously run code cell is a folium map, we can use that map as the starting map

In [4]:
%folium_map -m 52.02,-0.708,"My other marker" -z 14

To prevent a previous map being used as the starting map, set `-b None` or use the magic `%folium_new_map`.

We can also add sets of markers, provided in a variety of ways.

In [5]:
#We can accept all the following variants to get a marker in a map
markers={'lat':52.0250, 'lng':-0.7084,'popup':'Open University, Walton Hall'}
markers=[[52.0250, -0.7084,'Open University, Walton Hall'], [52., -0.7,'Open University, Walton Hall']]
markers=[52.0250, -0.7084,'Open University, Walton Hall']
markers=[{'lat':52.0250, 'lng':-0.7084,'popup':'Open University, Walton Hall'},
        {'lat':52.0, 'lng':-0.70,'popup':'Open University, Walton Hall'}]

In [6]:
#Lets make sure we use a fresh map
%folium_new_map -M markers

#### Adding Boundaries to Maps

Boundaries represented using a `.geojson` file can be rendered as an overlayed map layer.

The magic can also accept the path to a geoJSON file. If no latitude and longitude are selected, the centroid of the area will be used.

In [7]:
#Use the -b None flag to prevent the reuse of a previous map as the base map
%folium_map -b None -g boundaries/mk.json

#### Choropleth Maps

In [8]:
import pandas as pd
turnout=pd.read_csv('data/iw_turnout.csv')
turnout.head()

,Electoral Division,Candidate,Electorate,NoOnRoll,turnout
0,Arreton and Newchurch,Total votes cast,1286,3070,0.418893
1,Binstead and Fishbourne,Total votes cast,1160,2766,0.419378
2,"Brading, St Helens and Bembridge",Total votes cast,4640,6189,0.749717
3,Carisbrooke,Total votes cast,883,2632,0.335486
4,Central Wight,Total votes cast,1339,2904,0.461088


The magic can also plot chorpleths. The data to be used can be passed in by reference to the name of a variable containing the data to be plotted.

In [9]:
%folium_map -g boundaries/iw.json -d turnout -c 'Electoral Division','turnout' -k feature.properties.WD13NM

Or we can pass the path to a data file containing the data.

In [10]:
%folium_new_map -g boundaries/iw.json -d data/iw_turnout.csv -c 'Electoral Division','turnout' -k feature.properties.WD13NM

If we don't pass the GeoJSON key in, can we guess it by looking at overlaps between properties and the key column we want to use in the dataset?

(If there are spaces in the column name, we need to make sure the column name is given as a quoted string.)

In [11]:
%folium_new_map -g boundaries/iw.json -d turnout -c 'Electoral Division',turnout

If we are only provided with a data value column, can we guess a match for an area column in the data that matches an area property in the geojson file?

In [12]:
%folium_new_map -g boundaries/iw.json -d turnout -c 'turnout'

Also see if we can guess the data key column from the geojson key column.

In [13]:
%folium_new_map -g boundaries/iw.json -d data/iw_turnout.csv -c 'turnout' -k feature.properties.WD13NM

The last condition to consider is where we provide no information about the data value column either. Here, we could guess from one of the numerical columns, or return separate viewing layers for each numerical column?

### Helper magic to suggest column names

Provide some information about the geojson properties:

In [14]:
%geo_suggester -g boundaries/iw.json

Geojson - properties cols: WD13CD (['E05008483', 'E05008492', 'E05008490', '...']), WD13CDO (['00MWSJ', '00MWRN', '00MWRR', '...']), WD13NM (['Carisbrooke', 'Sandown South', 'Godshill and Wroxall', '...']), WD13NMW (['', '...'])


Provide some information about the data file columns:

In [15]:
%geo_suggester -d data/iw_turnout.csv

Data - numeric cols: Electorate, NoOnRoll, turnout
Data - object cols: Electoral Division, Candidate


Provide some information about the possible matches bewteen the data file columns and the geojson properties:

In [16]:
%geo_suggester -g boundaries/iw.json -d data/iw_turnout.csv

Data - numeric cols: Electorate, NoOnRoll, turnout
Data - object cols: Electoral Division, Candidate
Geojson - properties cols: WD13CD (['E05008483', 'E05008492', 'E05008490', '...']), WD13CDO (['00MWSJ', '00MWRN', '00MWRR', '...']), WD13NM (['Carisbrooke', 'Sandown South', 'Godshill and Wroxall', '...']), WD13NMW (['', '...'])
Possible matches between data and geojson: ('Electoral Division', 'feature.properties.WD13NM') (39)
